In [1]:
!pip install kaggle

Importing Dependencies

In [3]:
import os
import json

from zipfile import ZipFile
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data Collection - Kaggle API

In [18]:
kaggle_dataset = json.load(open('kaggle.json'))
os.environ['KAGGLE_USERNAME'] = kaggle_dataset['username']
os.environ['KAGGLE_KEY'] = kaggle_dataset['key']

In [19]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 479MB/s]


In [20]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [21]:
# We Will Unzip the File
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
  zip_ref.extractall()

 Load the Dataset

In [23]:
data = pd.read_csv('IMDB Dataset.csv')

In [24]:
data.shape

(50000, 2)

In [25]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [26]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [28]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [29]:
data.replace({'sentiment':{'positive':1, 'negative':0}}, inplace=True)

/tmp/ipython-input-29-202606334.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'sentiment':{'positive':1, 'negative':0}}, inplace=True)


In [30]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [31]:
data['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


Split data into training and test data

In [32]:
train_data, test_data= train_test_split(data, test_size=0.2, random_state=42)

In [33]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


Data Preprocessing

In [41]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)

In [44]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [50]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [51]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

In [52]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


LSTM = Long Short Term Memory

In [53]:
model = Sequential()
model.add(Embedding(5000, 128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [54]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [57]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [58]:
model.fit(X_train, Y_train, batch_size=64, epochs=10, validation_data=(X_test, Y_test))

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 377s 595ms/step - accuracy: 0.7382 - loss: 0.5115 - val_accuracy: 0.8401 - val_loss: 0.3792
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 383s 597ms/step - accuracy: 0.8624 - loss: 0.3356 - val_accuracy: 0.8756 - val_loss: 0.3009
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 380s 594ms/step - accuracy: 0.8855 - loss: 0.2847 - val_accuracy: 0.8815 - val_loss: 0.3065
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 373s 596ms/step - accuracy: 0.8940 - loss: 0.2641 - val_accuracy: 0.8827 - val_loss: 0.2887
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 382s 596ms/step - accuracy: 0.9045 - loss: 0.2372 - val_accuracy: 0.8880 - val_loss: 0.3110
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 379s 592ms/step - accuracy: 0.9227 - loss: 0.2020 - val_accuracy: 0.8927 - val_loss: 0.2828
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 383s 594ms/step - accuracy: 0.9322 - loss: 0.1781 - val_accuracy: 0.8970 - val_loss: 0.2871
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 381s 592ms/step - accuracy: 0.9410 -

In [59]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 84ms/step - accuracy: 0.8899 - loss: 0.3234
Test Loss: 0.322645366191864
Test Accuracy: 0.8920999765396118


In [60]:
def predict_sentiment(review):
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)

  prediction = model.predict(padded_sequence)
  sentiment = 'Positive' if prediction > 0.5 else 'Negative'

  return sentiment

In [61]:
# Example Usage
new_review = "This movie was fantastic! I loved every minute of it."
predicted_sentiment = predict_sentiment(new_review)
print(f'Predicted Sentiment: {predicted_sentiment}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 749ms/step
Predicted Sentiment: Positive


In [62]:
# Example 2 Usage
new_review = "The plot was confusing and the acting was terrible."
predicted_sentiment = predict_sentiment(new_review)
print(f'Predicted Sentiment: {predicted_sentiment}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Predicted Sentiment: Negative
